# Welcome to that-nlp-library

> Aim to be one of the most convenient library for common NLP tasks

## Install

```sh
pip install that_nlp_library
```

## What this library offers

TODO

## Tutorials

Visit [https://anhquan0412.github.io/that-nlp-library/](https://anhquan0412.github.io/that-nlp-library/)